In [3]:
import gensim.downloader
import pandas as pd
import numpy as np
import torch
from nltk.tokenize import word_tokenize
from sklearn.manifold import TSNE
from sklearn.metrics import accuracy_score
import datasets
import matplotlib.pyplot as plt
# enabling inline plots in Jupyter
%matplotlib inline
datasets.logging.set_verbosity_error()

# 1. Exploring word embeddings

1. Use [Gensim](https://radimrehurek.com/gensim/) (in particular, the `gensim.downloader.load()` module) to download and import 3 GloVe word embeddings: `glove-wiki-gigaword-200`, `glove-twitter-25`, `glove-twitter-200`.
2. What are the key differences in how these models were trained? How would you expect that to be reflected in their output?
3. Find the top 10 similar words to the word "republican" in all three models and print them. How similar are the models?
4. How many words in this neighborhood overlap between the two Twitter models, and the two 50-dimensional models?
5. Find which model has the closest distance between "republican" and "democrat".
6. Find what is the neighbor rank of "democrat" to "republican" in all three models. Does that look directly related to value of cosine similarity distance?

Note: if your computer is struggling, try substituting these models for smaller models.

# 2. Dimensionality reduction

Consider this paragraph:

```
The initial intelligence report, “Russia Likely to Denigrate Health of US Candidates to Influence 2020 Electoral Dynamics,” was proposed after an I&A employee “noticed an uptick in Russian state media efforts to question candidate Joseph Biden’s mental health. … The analyst believed foreign efforts questioning a candidate’s health were worth exploring because they could impact voters’ willingness to vote for that candidate and began drafting the product.”
```
Tokenize this text with a tokenizer of your choice and use the `glove_twitter_50` embeddings to create a T-SNE visualization of the words in this text. You are provided with a helper plotting function that will create a plot given the values for x and y axes, and the labels (words) corresponding to the dots on the graph.

Hint: you will need to create a matrix of Glove embeddings for the words in the text, which will then be processed by T-SNE. If any words are missing, their embeddings should be zero vectors.

In [9]:
def plot_with_matplotlib(x_vals, y_vals, labels, label_ratio=25):
    """plotting the x and y values for the decomposed embeddings
    label_ratio controls the percentage of the randomly sampled labels
    that will be labeled on the plot
    """
    import matplotlib.pyplot as plt
    import random

    # setting up the plot
    random.seed(0)
    plt.rcParams['font.size'] = '20'
    plt.figure(figsize=(12, 12))
    plt.scatter(x_vals, y_vals)

    # Labeling randomly subsampled data points (up to label_raion percentage of total points)
    indices = list(range(len(labels)))
    selected_indices = random.sample(indices, int(len(labels)*label_ratio/100))
    for i in selected_indices:
        plt.annotate(labels[i], (x_vals[i], y_vals[i]))

# 3. Torch embeddings

Torch module [torch.nn.EmbeddingBag](https://pytorch.org/docs/stable/generated/torch.nn.EmbeddingBag.html) provides a convenient way to provide pre-trained embeddings as inputs to linear layers. The embeddings can be randomly initialized or provided to the model from pre-trained weights.

1. Load the tweet_eval data. Create a full list of all tokens in the training and validation data (which should be tokenized and lowercased).
2. Create the embedding matrix for the list of words in the tweet_eval data that you created. The pre-trained weights should come from `glove_twitter_50` embeddings. Any missing words should be initialized as random embeddings. Your embedding matrix needs to contain one extra vector (all zeros).
3.  Create a version of FFN from the tutorial for the previous lecture which has an embedding layer of the model ([torch.nn.EmbeddingBag](https://pytorch.org/docs/stable/generated/torch.nn.EmbeddingBag.html).) That layer should be initialized with the above embedding matrix. Note the index of the extra all-zeros vector and pass it to EmbeddingBag as `padding_idx` parameter.


# 4. Preparing the data

1. Write a function turning its input text (a string) a vector of indices in total_vocabulary that corresponds to the tokenized words in the input text. For example, if the vector corresponding to "cat" is at index 5 in the embedding matrix, and the vector corresponding to "a" is at 130, then the phrase "a cat" would be represented as vector [130,5].
2. Use that function to transform the `tweet_eval` training and validation data into vectors of indices
3. Find the length of the longest tweet in training and validation data. Then you will need to "pad" all the tweet vectors to make them the same length as the longest tweet. Use the index of your extra zero vector in the embedding matrix as the padding token, and add the padding in the end of the sequence. For example, if the maximum sequence length is 5, and padding token is at index 0, then the phrase "a cat" would be represented as [130,5,0,0,0].
4. Modify the PyTorch dataset class you defined in the previous tutorial to receive this new type of vectorized training text. For the validation data, simply convert it to the PyTorch tensor.
5. Create the data loader for the training test.

# 5. Train the new model.

Train the model for 5 epochs in the same way as in the previous tutorial, with CrossEntropyLoss and Adam optimizer. Evaluate the trained model on the validation set.

Is it working better or worse than with TF-IDF vectorization?

# 6. Training your own word embeddings

1. Customize the provided data loader class to tokenize the input local text file with a tokenizer of your choice, lowercase it and remove the empty lines. The `__iter__` method must yield a list of tokens as the output.
2. Use the data loader class to load the provided raw text file of a several Trump's speeches during 2016 election campaign (`trump_speeches.txt`).
3. Use Gensim to train your own SkipGram embeddings, using this text as a corpus. Use vector size 300, window size 3, 3 epochs. Consider the parameters listed on Gensim's [word2vec reference page](https://radimrehurek.com/gensim/models/word2vec.html).
4. Compare the nearest neighbours of "China" and "Russia" in the Trump's word2vec. How different are they? Are these representations equally reliable? Why or why not?
5. Compare your results with your neighbor's results. Are they identical? Why do you think that is or is not the case?
6. Try retraining the model fixing the random seed to 42 and a single worker thread and compare the results with your neighbor again.
7. Save your trained model as a binary file.
8. Explore your Trump embeddings with any questions you may have yourself. E.g. what words are associated with "great" for him, and what words are associated with "bad"?
9. The size of the corpus is about 173 thousand tokens (about 1Mb as a plain text file). Based on this experiment, what would you say that word embedding technology allows you to do with textual data at this scale?

The speeches file comes from [this repository](https://github.com/ryanmcdermott/trump-speeches), distributed under "fair use".


In [25]:
# exercise template
class MyDataLoader(object):

    # initialize the corpus object for a given filename
    def __init__(self, filename):
        self.corpus = filename

    # you will need to define how you want the next chunk of text (document, sentence etc)
    # to be provided. Here, we assume one line per document format.
    def __iter__(self):
        for line in open(self.corpus, "r"):
            # check that the line is not empty:
            # tokenize and lowercase the string before yielding it
            # the output must be a list of tokens in the line
            yield line